In [22]:
import LiveStreamingEnv.env as env
import LiveStreamingEnv.load_trace as load_trace
import time
import numpy as np

bw_traces = './trace/test-xlow/'
video_size_files = ['./trace/video_trace/sports/AsianCup_China_Uzbekistan/frame_trace_',
                   './trace/video_trace/room/Fengtimo_2018_11_3/frame_trace_',
                   './trace/video_trace/game/YYF_2018_08_12/frame_trace_']
log_file_path = './log/'

DEBUG = False
DRAW = False


BIT_RATE = [500.0, 850.0, 1200.0, 1850.0] # kbps
TARGET_BUFFER = [2.0, 3.0] # seconds

all_cooked_time, all_cooked_bw, _ = load_trace.load_trace(bw_traces)

# default setting
last_bit_rate = 0
bit_rate = 0
target_buffer = 0
# QoE setting
reward_frame = 0
reward_gop = 0
reward_all = 0
SMOOTH_PENALTY = 0.02
REBUF_PENALTY = 1.5
LATENCY_PENALTY = 0.005
random_seed = 2

net_env = env.Environment(all_cooked_time=all_cooked_time,
                          all_cooked_bw=all_cooked_bw,
                          random_seed=random_seed,
                          logfile_path=log_file_path,
                          VIDEO_SIZE_FILE=video_size_files[0],
                          Debug = DEBUG)

In [23]:
frame_time_len = 0.04
reward_gop = 0.0
reward_sum = 0.0
while True:
    time, time_interval, send_data_size, chunk_len, rebuf, buffer_size, play_time_len,end_delay, \
    cdn_newest_id, download_id, cdn_has_frame, decision_flag, buffer_flag,cdn_flag, end_of_video = \
    net_env.get_video_frame(bit_rate,target_buffer)
    
    if not cdn_flag:
        reward_frame = frame_time_len * float(BIT_RATE[bit_rate]) / 1000  - REBUF_PENALTY * rebuf - LATENCY_PENALTY  * end_delay
    else:
        reward_frame = -(REBUF_PENALTY * rebuf)
    reward_gop += reward_frame
    reward_sum += reward_frame
    if decision_flag or end_of_video:
        # reward formate = play_time * BIT_RATE - 4.3 * rebuf - 1.2 * end_delay
        reward_frame = -1 * SMOOTH_PENALTY * (abs(BIT_RATE[bit_rate] - BIT_RATE[last_bit_rate]) / 1000)
        reward_gop += reward_frame
        reward_sum += reward_frame
        # last_bit_rate
        last_bit_rate = bit_rate
        bit_rate, target_buffer = 0, 0
        print('time: %.3f, reward for current GoP is: %.5f' % (time, reward_gop))
        reward_gop = 0.0
#     print('Reward for current frame is: %.5f' % (reward_frame))
    
    if end_of_video or time >= 1500:
        print('sum of reward is: %.3f' % (reward_sum))
        break

time: 1.526, reward for current GoP is: -1.98419
time: 2.682, reward for current GoP is: -0.17420
time: 4.123, reward for current GoP is: 0.15599
time: 6.048, reward for current GoP is: 0.17629
time: 8.045, reward for current GoP is: 0.20011
time: 10.273, reward for current GoP is: 0.22341
time: 12.108, reward for current GoP is: 0.23308
time: 14.165, reward for current GoP is: 0.26710
time: 16.176, reward for current GoP is: 0.28013
time: 18.219, reward for current GoP is: 0.28911
time: 20.312, reward for current GoP is: 0.30139
time: 22.305, reward for current GoP is: 0.30944
time: 24.300, reward for current GoP is: 0.31034
time: 26.305, reward for current GoP is: 0.30922
time: 28.383, reward for current GoP is: 0.32936
time: 30.538, reward for current GoP is: 0.33089
time: 33.225, reward for current GoP is: 0.34377
time: 35.033, reward for current GoP is: 0.35030
time: 36.751, reward for current GoP is: 0.34895
time: 38.510, reward for current GoP is: 0.35699
time: 40.575, reward fo

time: 1085.326, reward for current GoP is: 0.23907
time: 1087.314, reward for current GoP is: 0.29287
time: 1089.609, reward for current GoP is: 0.33080
time: 1091.730, reward for current GoP is: 0.37119
time: 1093.730, reward for current GoP is: 0.39516
time: 1095.749, reward for current GoP is: 0.40084
time: 1097.821, reward for current GoP is: 0.40661
time: 1099.836, reward for current GoP is: 0.42050
time: 1102.294, reward for current GoP is: 0.47600
time: 1104.303, reward for current GoP is: 0.52228
time: 1106.226, reward for current GoP is: 0.52059
time: 1108.239, reward for current GoP is: 0.52050
time: 1110.394, reward for current GoP is: 0.55962
time: 1112.478, reward for current GoP is: 0.57636
time: 1114.568, reward for current GoP is: 0.59654
time: 1116.781, reward for current GoP is: 0.61700
time: 1118.818, reward for current GoP is: 0.66022
time: 1120.849, reward for current GoP is: 0.67096
time: 1123.019, reward for current GoP is: 0.68991
time: 1125.011, reward for curr

## Test EnvWrap

In [ ]:
from env_wrap import EnvWrap
from env_args import EnvArgs
args = EnvArgs()
env = EnvWrap(args, video_file_id=0)
print('import')

In [ ]:


reward_frame = 0
reward_all = 0
reward_gop = 0
env.reset()
while True:
    reward_frame, end_of_video, decision_flag = env.step_gop(action=0)
#     print('reward frame is %.6f' % (reward_frame))
    if not decision_flag:
        reward_all += reward_frame
    if decision_flag or end_of_video:
        reward_gop = env.get_reward_gop()
#         print('reward gop is: %.6f' % (reward_gop))
#         print('check reward gop: %.6f' % (reward_all))
        state_gop = env.get_state_gop()
#         print(state_gop[0:1, -1]) # time [0, 1]
#         print(state_gop[1:2, -1]) # current buffer size [0, 10] 
#         print(state_gop[2:3, -1]) # last bitrate 
        print('thp: ', state_gop[3:4, -1]) # throughput for last gop [0, 10] // need considering finer level thp like every 5 frames
#         print(state_gop[4:5, -1]) # gop rebuf [0, 10]
        print('download time', state_gop[5:6, -1]) # gop download time [0, 10]
#         print(state_gop[6:7, -1]) # gop play time len [0, 10]
#         print(state_gop[7:8, -1]) # gop delay (ms -> 100ms)[]
#         print(state_gop[8:9, -1]) # rebuffering event ratio (/10)
#         print(state_gop[9:10, -1]) # cdn rebuffering event ratio (/10)
#         print(state_gop[10:11, :4]) # next gop sizes [0, 10]
#         print(state_gop[11:12, -1]) # frame skip
        reward_all = reward_frame
        print('reward gop: ', reward_gop)
    if end_of_video:
        env.reset()
        reward_frame = 0
        reward_all = 0
        reward_gop = 0
        print('end of video')
        break

In [ ]:
import numpy as np

In [ ]:
a = [[1,2,3,4], [5,6,7,8], [1,4,5,6], [2,3,4,5]]

In [ ]:
a = np.array(a)

In [ ]:
print(a)

In [ ]:
I_frame = a[:4, 0]

In [ ]:
print(I_frame)

In [ ]:
gop = np.sum(a, axis=1)

In [ ]:
print(gop)

In [ ]:
p = a[:4, 1:-1]
print(p)

In [ ]:
p_ave = np.mean(p, axis=1)

In [ ]:
print(p_ave)

In [ ]:
next_gop = I_frame + p_ave * 49

In [ ]:
print(next_gop)

In [ ]:
gop_end_id = 0
for i in range(1, 12):
    if i > 10:
        gop_end_id = i
        break

In [ ]:
print(gop_end_id)

In [ ]:
print(type(gop_end_id))